In [1]:
monitor='val_accuracy'
epochs=30
batch_size=16
input_shape=(128, 128, 3) # please resize it to (224,224,3) if you have enough RAM
Verbose=True

This source code requires a **HIGH RAM** machine.

You might need to install this on your system:

apt-get install python3-opencv git

In [2]:
import os

if not os.path.isdir('k'):
  !git clone -b development14 https://github.com/joaopauloschuler/k-neural-api.git k
else:
  !cd k && git pull

!cd k && pip install .

Cloning into 'k'...
remote: Enumerating objects: 1400, done.
remote: Counting objects: 100% (849/849), done.
remote: Compressing objects: 100% (548/548), done.
remote: Total 1400 (delta 624), reused 519 (delta 300), pack-reused 551
Receiving objects: 100% (1400/1400), 15.00 MiB | 25.19 MiB/s, done.
Resolving deltas: 100% (980/980), done.
Processing /content/k
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for cai: filename=cai-0.1.5-py3-none-any.whl size=57413 sha256=d2f36131ead1e20489fbedfcc846205b3745683a8f784a9cb952b56ac91db311
  Stored in directory: /tmp/pip-ephem-wheel-cache-d5agk42l/wheels/c1/8a/57/56dbba25eff58e52e536543

In [3]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

import skimage
print('skimage version',  skimage.__version__)
import tensorflow as tf
print(tf.__version__)
import sys
import cai
import cai.datasets
import cai.densenet
import cai.util
import cai.models

Python version
3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
Version info.
sys.version_info(major=3, minor=7, micro=12, releaselevel='final', serial=0)
skimage version 0.18.3
2.7.0


In [4]:
url_zip_file="https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded"
local_zip_file="plant_leaf.zip"
expected_folder_name="plant_leaf"
cai.datasets.download_zip_and_extract(
    url_zip_file=url_zip_file, local_zip_file=local_zip_file, 
    expected_folder_name=expected_folder_name, Verbose=Verbose)

Downloading:  https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded  to  plant_leaf.zip
Decompressing into:  plant_leaf


In [5]:
import random
import os
import multiprocessing
import glob
import numpy as np
import tensorflow
from sklearn.model_selection import train_test_split
import sklearn.utils.class_weight
from tensorflow import keras
from tensorflow.keras import mixed_precision
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import utils as np_utils
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
!rm -r plant_leaf/Plant_leave_diseases_dataset_without_augmentation/Background_without_leaves -R
data_dir = "plant_leaf/Plant_leave_diseases_dataset_without_augmentation/"
print(os.listdir(data_dir))

['Pepper,_bell___Bacterial_spot', 'Blueberry___healthy', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Peach___healthy', 'Apple___Black_rot', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Potato___Early_blight', 'Corn___healthy', 'Corn___Common_rust', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Late_blight', 'Raspberry___healthy', 'Grape___Esca_(Black_Measles)', 'Tomato___Target_Spot', 'Grape___healthy', 'Tomato___healthy', 'Apple___Apple_scab', 'Apple___healthy', 'Grape___Black_rot', 'Corn___Northern_Leaf_Blight', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Tomato___Septoria_leaf_spot', 'Potato___Late_blight', 'Squash___Powdery_mildew', 'Peach___Bacterial_spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___Early_blight', 'Cherry___healthy', 'Potato___healthy', 'Pepper,_bell___healthy', 'Cherry___Powdery_mildew', 'Tomato___Bacterial_spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Soybean___healthy', 'Tomato___Leaf_Mold', 'Apple___Cedar_apple_rust', 'Orange___Ha

In [7]:
train_x, val_x, test_x, train_y, val_y, test_y, classweight, classes = cai.datasets.load_images_from_folders(seed=7, root_dir=data_dir, lab=True, 
  verbose=Verbose, bipolar=False, base_model_name='plant_leaf',
  training_size=0.6, validation_size=0.2, test_size=0.2,
  target_size=(input_shape[0],input_shape[1]), 
  has_training=True, has_validation=True, has_testing=True, 
  smart_resize=True)

Loading  38  classes.
smart resize is enabled.
loading train images
train shape is: (32571, 128, 128, 3)
loading validation images
validation shape is: (10858, 128, 128, 3)
loading test images
test shape is: (10876, 128, 128, 3)
Channel  0  min: 0.0  max: 1.0
Channel  1  min: 0.19593212  max: 0.88375795
Channel  2  min: 0.24579802  max: 0.9470912
Loaded.


In [8]:
print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

(32571, 128, 128, 3) (10858, 128, 128, 3) (10876, 128, 128, 3)
(32571, 38) (10858, 38) (10876, 38)


In [9]:
for two_paths_second_block in [False]:
  for l_ratio in [0.2]:
    basefilename = 'two-path-inception-v2.7-'+str(two_paths_second_block)+'-'+str(l_ratio)
    print('Running: '+basefilename)
    model = cai.models.compiled_two_path_inception_v3(
      input_shape=input_shape,
      classes=38, 
      two_paths_first_block=True,
      two_paths_second_block=two_paths_second_block,
      l_ratio=l_ratio,
      ab_ratio=(1-l_ratio),
      max_mix_idx=5, 
      model_name='two_path_inception_v3'
      )    
    monitor='val_accuracy'
    best_result_file_name = basefilename+'-best_result.hdf5'
    save_best = keras.callbacks.ModelCheckpoint(
      filepath=best_result_file_name, 
      monitor=monitor, 
      verbose=1, 
      save_best_only=True, 
      save_weights_only=False, 
      mode='max', 
      period=1)
    history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size,
      validation_data=(val_x,val_y),
      callbacks=[save_best],class_weight=classweight,
      workers=multiprocessing.cpu_count())
    print('Testing Last Model: '+basefilename)
    evaluated = model.evaluate(test_x,test_y)
    for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
      print(name,metric)
    print('Best Model Results: '+basefilename)
    model = keras.models.load_model(best_result_file_name, custom_objects={'CopyChannels': cai.layers.CopyChannels})
    evaluated = model.evaluate(test_x,test_y)
    cai.models.save_model(model, basefilename)
    for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
      print(name,metric)
    print('Finished: '+basefilename)

Running: two-path-inception-v2.7-False-0.2


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/30
2036/2036 [==============================] - ETA: 0s - loss: 1.4251 - accuracy: 0.5848
Epoch 00001: val_accuracy improved from -inf to 0.26064, saving model to two-path-inception-v2.7-False-0.2-best_result.hdf5
2036/2036 [==============================] - 71s 29ms/step - loss: 1.4251 - accuracy: 0.5848 - val_loss: 3.8903 - val_accuracy: 0.2606
Epoch 2/30
2036/2036 [==============================] - ETA: 0s - loss: 0.5014 - accuracy: 0.8291
Epoch 00002: val_accuracy improved from 0.26064 to 0.89722, saving model to two-path-inception-v2.7-False-0.2-best_result.hdf5
2036/2036 [==============================] - 57s 28ms/step - loss: 0.5014 - accuracy: 0.8291 - val_loss: 0.3199 - val_accuracy: 0.8972
Epoch 3/30
2036/2036 [==============================] - ETA: 0s - loss: 0.2957 - accuracy: 0.8942
Epoch 00003: val_accuracy did not improve from 0.89722
2036/2036 [==============================] - 55s 27ms/step - loss: 0.2957 - accuracy: 0.8942 - val_loss: 0.3747 - val_accuracy: 0.